In [ ]:
import xarray as xr
!pip install cftime  # Install cftime to handle non-standard calendars
# Install required libraries
!pip install xarray cfgrib h5py netCDF4


# Load the file
from netCDF4 import Dataset

nc_file = Dataset('/content/drive/MyDrive/Colab Notebooks/file_4.nc', 'r')

# Explore the file
# print(nc_file)
print(nc_file.variables.keys())  # List all variables

# Access specific variables
temperature = nc_file.variables['ts'][:]  # Replace 'temperature' with the variable name
print(temperature)